In [1]:
import pandas as pd
import numpy as np
import re
import scattertext as st
import spacy
import pickle
from elasticsearch import Elasticsearch, helpers

In [2]:
df_parsed = pd.read_pickle('df_parsed.pkl')

In [5]:
df_parsed.columns

Index(['Message-ID', 'Date', 'From', 'To', 'Subject', 'Mime-Version',
       'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc',
       'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Body', 'Bcc', 'Cc',
       'file', 'Date_'],
      dtype='object')

In [3]:
es = Elasticsearch()

In [7]:
es.indices.delete('enron')

{'acknowledged': True}

In [8]:
es.indices.create('enron')

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'enron'}

In [6]:
def get_rows(df):
    for i, df_t in df.iterrows():
        if pd.isnull(df_t['Date_']):
            d = 'null'
        else:
            d = df_t['Date_'].strftime('%Y-%m-%dT%H:%M:%S')
        yield {'_index': 'enron', '_type': '_doc', '_source': {'Date': d,
                                                               'From': df_t.From,
                                                               'To': df_t.To,
                                                               'Subject': df_t.Subject,
                                                               'Body': df_t.Body}}

In [9]:
df_parsed_sample = df_parsed.sample(100000, random_state=42)

In [10]:
r = get_rows(df_parsed_sample)

In [11]:
helpers.bulk(es, r)

(100000, [])